In [1]:
import pandas as pd
import geopandas as gpd
import opendatasets as od
import requests
import numpy as np

In [4]:
od.download('https://datahub.io/core/geo-countries/r/countries.geojson')
countries_geojson = gpd.read_file('countries.geojson')
response = requests.get('https://api.worldbank.org/v2/en/indicator/NY.GDP.MKTP.CD?downloadformat=excel')
output = open('GDP.xls', 'wb')
output.write(response.content)
output.close()
GDP = pd.read_excel('GDP.xls', skiprows=[0,1,2], engine='xlrd')

response = requests.get('https://api.worldbank.org/v2/en/indicator/SP.POP.TOTL?downloadformat=excel')
output = open('Population.xls', 'wb')
output.write(response.content)
output.close()
Population = pd.read_excel('Population.xls', skiprows=[0,1,2], engine='xlrd')

Using downloaded and verified file: .\countries.geojson


In [2]:
rampen_df = pd.read_csv('emdat_public_2022_11_02_query_uid-DNpCZl.csv', sep=';')
rampen_df = rampen_df[rampen_df.Year >= 1961].reset_index(drop=True).fillna(0)
rampen_df = rampen_df[rampen_df.Year <= 2021].reset_index(drop=True).fillna(0)
rampen_df['Total Affected new'] = rampen_df['No Affected']+rampen_df['No Injured']+rampen_df['No Homeless']


rampen_df_controle = rampen_df.groupby(['ISO', 'Country', 'Year', 'Disaster Group', 'Disaster Subgroup', 'Disaster Type',
       'Disaster Subtype'])['Total Affected new'].sum().reset_index()
rampen_df_controle2 = rampen_df.groupby(['ISO', 'Country', 'Year', 'Disaster Group', 'Disaster Subgroup', 'Disaster Type',
       'Disaster Subtype'])['Total Deaths'].sum().reset_index()

rampen_df = rampen_df_controle2
rampen_df['Total Affected new'] = rampen_df_controle['Total Affected new']

rampen_df['Disaster Subtype'] = rampen_df.apply(lambda row: 'NaN - ' + row['Disaster Type'] if (row['Disaster Subtype']==0) else row['Disaster Subtype'], axis=1)

rampen_df['Jaar 0']=0
rampen_df['Jaar 1']=0
rampen_df['Jaar 2']=0
rampen_df['Jaar 3']=0


for index, row in rampen_df.iterrows():
    if len(GDP[GDP['Country Code']==rampen_df['ISO'][index]][str(rampen_df['Year'][index])])>0:
        if rampen_df['Year'][index] <=2021:
            rampen_df.iloc[index, rampen_df.columns.get_loc('Jaar 0')] = \
            (GDP[GDP['Country Code']==rampen_df['ISO'][index]][str(rampen_df['Year'][index])].values[0]\
             - GDP[GDP['Country Code']==rampen_df['ISO'][index]][str(rampen_df['Year'][index]-1)].values[0])/\
            GDP[GDP['Country Code']==rampen_df['ISO'][index]][str(rampen_df['Year'][index]-1)].values[0]\
             - (GDP[GDP['Country Code']=='WLD'][str(rampen_df['Year'][index])].values[0]\
             - GDP[GDP['Country Code']=='WLD'][str(rampen_df['Year'][index]-1)].values[0])/\
            GDP[GDP['Country Code']=='WLD'][str(rampen_df['Year'][index]-1)].values[0]
        
        
        if rampen_df['Year'][index]+1 <= 2021:
            rampen_df.iloc[index, rampen_df.columns.get_loc('Jaar 1')] = \
            (GDP[GDP['Country Code']==rampen_df['ISO'][index]][str(rampen_df['Year'][index]+1)].values[0]\
             - GDP[GDP['Country Code']==rampen_df['ISO'][index]][str(rampen_df['Year'][index])].values[0])/\
            GDP[GDP['Country Code']==rampen_df['ISO'][index]][str(rampen_df['Year'][index])].values[0]\
            - (GDP[GDP['Country Code']=='WLD'][str(rampen_df['Year'][index]+1)].values[0]\
             - GDP[GDP['Country Code']=='WLD'][str(rampen_df['Year'][index])].values[0])/\
            GDP[GDP['Country Code']=='WLD'][str(rampen_df['Year'][index])].values[0]
        
        if rampen_df['Year'][index]+2 <=2021:   
            rampen_df.iloc[index, rampen_df.columns.get_loc('Jaar 2')] = \
            (GDP[GDP['Country Code']==rampen_df['ISO'][index]][str(rampen_df['Year'][index]+2)].values[0]\
             - GDP[GDP['Country Code']==rampen_df['ISO'][index]][str(rampen_df['Year'][index]+1)].values[0])/\
            GDP[GDP['Country Code']==rampen_df['ISO'][index]][str(rampen_df['Year'][index]+1)].values[0]\
             - (GDP[GDP['Country Code']=='WLD'][str(rampen_df['Year'][index]+2)].values[0]\
             - GDP[GDP['Country Code']=='WLD'][str(rampen_df['Year'][index]+1)].values[0])/\
            GDP[GDP['Country Code']=='WLD'][str(rampen_df['Year'][index]+1)].values[0]
        
        if rampen_df['Year'][index]+3 <=2021:    
            rampen_df.iloc[index, rampen_df.columns.get_loc('Jaar 3')] = \
            (GDP[GDP['Country Code']==rampen_df['ISO'][index]][str(rampen_df['Year'][index]+3)].values[0]\
             - GDP[GDP['Country Code']==rampen_df['ISO'][index]][str(rampen_df['Year'][index]+2)].values[0])/\
            GDP[GDP['Country Code']==rampen_df['ISO'][index]][str(rampen_df['Year'][index]+2)].values[0]\
             - (GDP[GDP['Country Code']=='WLD'][str(rampen_df['Year'][index]+3)].values[0]\
             - GDP[GDP['Country Code']=='WLD'][str(rampen_df['Year'][index]+2)].values[0])/\
            GDP[GDP['Country Code']=='WLD'][str(rampen_df['Year'][index]+2)].values[0]

rampen_df['Jaar 0']=rampen_df['Jaar 0']*100
rampen_df['Jaar 1']=rampen_df['Jaar 1']*100
rampen_df['Jaar 2']=rampen_df['Jaar 2']*100
rampen_df['Jaar 3']=rampen_df['Jaar 3']*100


print(rampen_df)
rampen_df.to_csv('rampen_df.csv', index=False)

NameError: name 'GDP' is not defined